Imports

In [157]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import json
import time
import feedparser
import datetime
import sys
import warnings
import re
import bs4
import nltk
import yfinance as yf
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation
from heapq import nlargest
import ssl 
from collections import defaultdict
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings('ignore')

In [158]:

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/mohammedal-
[nltk_data]     rasheed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Constants

In [159]:
ARTICLES_LINK = "https://finance.yahoo.com/rss/topstories"

Get Todays Articles

In [160]:
# for each link in the dataframe, get the article text
def get_article_text(link):
    """
    Get the article text from the link.
    """
    try:
        res = requests.get(link)
        if res.status_code != 200:
            raise Exception("Error: API request unsuccessful.")
        soup = bs4.BeautifulSoup(res.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = ' '.join([p.text for p in paragraphs])
    except:
        text = ''
    return text

def get_articles():
    """
    Get the top 10 articles from Yahoo Finance.
    """
    df = pd.DataFrame(columns=['title', 'link', 'summary', 'published'])
    res = requests.get(ARTICLES_LINK)
    if res.status_code != 200:
        raise Exception("Error: API request unsuccessful.")
    data = feedparser.parse(res.text)
    articles = data.entries
    for article in articles:
        row = pd.Series(article)
        df = df.append(row, ignore_index=True)
    df['text'] = df["link"].apply(get_article_text)
    return df

df = get_articles()
df.head()

,title,link,summary,published,title_detail,links,published_parsed,source,id,guidislink,media_content,media_credit,credit,text
0,"Chevron, Comcast, and 10 More Companies That Raised Their Dividends This Week",https://www.barrons.com/articles/chevron-comcast-blackrock-dividend-stocks-51674848026?siteid=yhoof2&yptr=yahoo,NaN,2023-01-28T09:00:00Z,"{'type': 'text/plain', 'language': None, 'base': '', 'value': 'Chevron, Comcast, and 10 More Companies That Raised Their Dividends This Week'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://www.barrons.com/articles/chevron-comcast-blackrock-dividend-stocks-51674848026?siteid=yhoof2&yptr=yahoo'}]","(2023, 1, 28, 9, 0, 0, 5, 28, 0)","{'href': 'http://www.barrons.com/', 'title': 'Barrons.com'}",chevron-comcast-blackrock-dividend-stocks-51674848026?siteid=yhoof2&yptr=yahoo,False,"[{'height': '86', 'url': 'https://media.zenfs.com/en/Barrons.com/b1b28190c68c7cdff1823fd3f814da49', 'width': '130'}]",[{'role': 'publishing company'}],,
1,This janitor in Vermont amassed an $8M fortune without anyone around him knowing. Here are the 3 simple techniques that made Ronald Read rich — and can do the same for you,https://finance.yahoo.com/news/janitor-vermont-amassed-8m-fortune-140000770.html,NaN,2023-01-28T14:00:00Z,"{'type': 'text/plain', 'language': None, 'base': '', 'value': 'This janitor in Vermont amassed an $8M fortune without anyone around him knowing. Here are the 3 simple techniques that made Ronald R...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://finance.yahoo.com/news/janitor-vermont-amassed-8m-fortune-140000770.html'}]","(2023, 1, 28, 14, 0, 0, 5, 28, 0)","{'href': 'https://moneywise.com/', 'title': 'MoneyWise'}",janitor-vermont-amassed-8m-fortune-140000770.html,False,"[{'height': '86', 'url': 'https://media.zenfs.com/en/moneywise_327/c48d4f22d5c1eae10c335a74b4a4149f', 'width': '130'}]",[{'role': 'publishing company'}],,"Warren Buffett is reported to have once said, “You don't need to have extraordinary effort to achieve extraordinary results. You just need to do the ordinary, everyday things exceptionally well.” ..."
2,New 'PELOSI' bill may have a spiteful name — but there's still 'tremendous momentum' in the crusade to stop politicians from insider trading. Here's why,https://finance.yahoo.com/news/american-people-tired-lawmakers-introduce-150000544.html,NaN,2023-01-27T21:30:00Z,"{'type': 'text/plain', 'language': None, 'base': '', 'value': 'New 'PELOSI' bill may have a spiteful name — but there's still 'tremendous momentum' in the crusade to stop politicians from insider ...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://finance.yahoo.com/news/american-people-tired-lawmakers-introduce-150000544.html'}]","(2023, 1, 27, 21, 30, 0, 4, 27, 0)","{'href': 'https://moneywise.com/', 'title': 'MoneyWise'}",american-people-tired-lawmakers-introduce-150000544.html,False,"[{'height': '86', 'url': 'https://media.zenfs.com/en/moneywise_327/c6f22b4b672364c0f325801977b24ec5', 'width': '130'}]",[{'role': 'publishing company'}],,"In a mic drop moment in the debate around insider trading in Congress, Sen. Josh Hawley of Missouri has introduced the PELOSI Act – trolling former Speaker of the House Nancy Pelosi. With his acro..."
3,Lucid stock soars amid Saudi buyout speculation,https://finance.yahoo.com/news/lucid-stock-soars-amid-saudi-buyout-speculation-212055245.html,NaN,2023-01-27T21:20:55Z,"{'type': 'text/plain', 'language': None, 'base': '', 'value': 'Lucid stock soars amid Saudi buyout speculation'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://finance.yahoo.com/news/lucid-stock-soars-amid-saudi-buyout-speculation-212055245.html'}]","(2023, 1, 27, 21, 20, 55, 4, 27, 0)","{'href': 'http://finance.yahoo.com/', 'title': 'Yahoo Finance'}",lucid-stock-soars-amid-saudi-buyout-speculation-212055245.html,False,"[{'height': '86', 'url': 'https://s.yimg.com/os/creatr-uploaded-images/2023-01/b5c7bf90-9e81-11ed-bfac-29409587cd80', 'width': '130'}]",[

Data Cleaning

In [161]:
# drop summary row
df = df.drop(columns=['summary'])
df = df.drop(columns=['links', 'title_detail', 'published_parsed', 'source', 'id', 'guidislink'])
df = df.drop(columns=['media_content', 'media_credit', 'credit'])

df.head()

,title,link,published,text
0,"Chevron, Comcast, and 10 More Companies That Raised Their Dividends This Week",https://www.barrons.com/articles/chevron-comcast-blackrock-dividend-stocks-51674848026?siteid=yhoof2&yptr=yahoo,2023-01-28T09:00:00Z,
1,This janitor in Vermont amassed an $8M fortune without anyone around him knowing. Here are the 3 simple techniques that made Ronald Read rich — and can do the same for you,https://finance.yahoo.com/news/janitor-vermont-amassed-8m-fortune-140000770.html,2023-01-28T14:00:00Z,"Warren Buffett is reported to have once said, “You don't need to have extraordinary effort to achieve extraordinary results. You just need to do the ordinary, everyday things exceptionally well.” ..."
2,New 'PELOSI' bill may have a spiteful name — but there's still 'tremendous momentum' in the crusade to stop politicians from insider trading. Here's why,https://finance.yahoo.com/news/american-people-tired-lawmakers-introduce-150000544.html,2023-01-27T21:30:00Z,"In a mic drop moment in the debate around insider trading in Congress, Sen. Josh Hawley of Missouri has introduced the PELOSI Act – trolling former Speaker of the House Nancy Pelosi. With his acro..."
3,Lucid stock soars amid Saudi buyout speculation,https://finance.yahoo.com/news/lucid-stock-soars-amid-saudi-buyout-speculation-212055245.html,2023-01-27T21:20:55Z,Lucid (LCID) shares soared as much as 88% on Friday and were repeatedly halted for volatility following speculation that Saudi Arabia's Public Investment Fund (PIF) is considering buying a remaini...
4,Billionaire Adani's Empire Loses $51 Billion in 48 Hours,https://www.thestreet.com/technology/billionaire-adanis-empire-loses-51-billion-in-48-hours?puc=yahoo&cm_ven=YAHOO&yptr=yahoo,2023-01-27T14:17:00Z,"Suspicion is creating chaos in the empire built by the Indian billionaire Gautam Adani, who since last year has become the richest man in Asia. Adani, 60, saw his fortune jump by more than $40 bi..."


In [162]:
df['published'] = str(df['published']).split('T')[0]
df.head()

,title,link,published,text
0,"Chevron, Comcast, and 10 More Companies That Raised Their Dividends This Week",https://www.barrons.com/articles/chevron-comcast-blackrock-dividend-stocks-51674848026?siteid=yhoof2&yptr=yahoo,0 2023-01-28,
1,This janitor in Vermont amassed an $8M fortune without anyone around him knowing. Here are the 3 simple techniques that made Ronald Read rich — and can do the same for you,https://finance.yahoo.com/news/janitor-vermont-amassed-8m-fortune-140000770.html,0 2023-01-28,"Warren Buffett is reported to have once said, “You don't need to have extraordinary effort to achieve extraordinary results. You just need to do the ordinary, everyday things exceptionally well.” ..."
2,New 'PELOSI' bill may have a spiteful name — but there's still 'tremendous momentum' in the crusade to stop politicians from insider trading. Here's why,https://finance.yahoo.com/news/american-people-tired-lawmakers-introduce-150000544.html,0 2023-01-28,"In a mic drop moment in the debate around insider trading in Congress, Sen. Josh Hawley of Missouri has introduced the PELOSI Act – trolling former Speaker of the House Nancy Pelosi. With his acro..."
3,Lucid stock soars amid Saudi buyout speculation,https://finance.yahoo.com/news/lucid-stock-soars-amid-saudi-buyout-speculation-212055245.html,0 2023-01-28,Lucid (LCID) shares soared as much as 88% on Friday and were repeatedly halted for volatility following speculation that Saudi Arabia's Public Investment Fund (PIF) is considering buying a remaini...
4,Billionaire Adani's Empire Loses $51 Billion in 48 Hours,https://www.thestreet.com/technology/billionaire-adanis-empire-loses-51-billion-in-48-hours?puc=yahoo&cm_ven=YAHOO&yptr=yahoo,0 2023-01-28,"Suspicion is creating chaos in the empire built by the Indian billionaire Gautam Adani, who since last year has become the richest man in Asia. Adani, 60, saw his fortune jump by more than $40 bi..."


In [163]:
# delete the 0 in the beginning of the date
df['published'] = df['published'].str[1:]
df.head()

,title,link,published,text
0,"Chevron, Comcast, and 10 More Companies That Raised Their Dividends This Week",https://www.barrons.com/articles/chevron-comcast-blackrock-dividend-stocks-51674848026?siteid=yhoof2&yptr=yahoo,2023-01-28,
1,This janitor in Vermont amassed an $8M fortune without anyone around him knowing. Here are the 3 simple techniques that made Ronald Read rich — and can do the same for you,https://finance.yahoo.com/news/janitor-vermont-amassed-8m-fortune-140000770.html,2023-01-28,"Warren Buffett is reported to have once said, “You don't need to have extraordinary effort to achieve extraordinary results. You just need to do the ordinary, everyday things exceptionally well.” ..."
2,New 'PELOSI' bill may have a spiteful name — but there's still 'tremendous momentum' in the crusade to stop politicians from insider trading. Here's why,https://finance.yahoo.com/news/american-people-tired-lawmakers-introduce-150000544.html,2023-01-28,"In a mic drop moment in the debate around insider trading in Congress, Sen. Josh Hawley of Missouri has introduced the PELOSI Act – trolling former Speaker of the House Nancy Pelosi. With his acro..."
3,Lucid stock soars amid Saudi buyout speculation,https://finance.yahoo.com/news/lucid-stock-soars-amid-saudi-buyout-speculation-212055245.html,2023-01-28,Lucid (LCID) shares soared as much as 88% on Friday and were repeatedly halted for volatility following speculation that Saudi Arabia's Public Investment Fund (PIF) is considering buying a remaini...
4,Billionaire Adani's Empire Loses $51 Billion in 48 Hours,https://www.thestreet.com/technology/billionaire-adanis-empire-loses-51-billion-in-48-hours?puc=yahoo&cm_ven=YAHOO&yptr=yahoo,2023-01-28,"Suspicion is creating chaos in the empire built by the Indian billionaire Gautam Adani, who since last year has become the richest man in Asia. Adani, 60, saw his fortune jump by more than $40 bi..."


Analyzing Data

In [164]:
# get length of texts
df['text_length'] = df['text'].str.len()
df.head()

,title,link,published,text,text_length
0,"Chevron, Comcast, and 10 More Companies That Raised Their Dividends This Week",https://www.barrons.com/articles/chevron-comcast-blackrock-dividend-stocks-51674848026?siteid=yhoof2&yptr=yahoo,2023-01-28,,0
1,This janitor in Vermont amassed an $8M fortune without anyone around him knowing. Here are the 3 simple techniques that made Ronald Read rich — and can do the same for you,https://finance.yahoo.com/news/janitor-vermont-amassed-8m-fortune-140000770.html,2023-01-28,"Warren Buffett is reported to have once said, “You don't need to have extraordinary effort to achieve extraordinary results. You just need to do the ordinary, everyday things exceptionally well.” ...",4369
2,New 'PELOSI' bill may have a spiteful name — but there's still 'tremendous momentum' in the crusade to stop politicians from insider trading. Here's why,https://finance.yahoo.com/news/american-people-tired-lawmakers-introduce-150000544.html,2023-01-28,"In a mic drop moment in the debate around insider trading in Congress, Sen. Josh Hawley of Missouri has introduced the PELOSI Act – trolling former Speaker of the House Nancy Pelosi. With his acro...",5610
3,Lucid stock soars amid Saudi buyout speculation,https://finance.yahoo.com/news/lucid-stock-soars-amid-saudi-buyout-speculation-212055245.html,2023-01-28,Lucid (LCID) shares soared as much as 88% on Friday and were repeatedly halted for volatility following speculation that Saudi Arabia's Public Investment Fund (PIF) is considering buying a remaini...,2198
4,Billionaire Adani's Empire Loses $51 Billion in 48 Hours,https://www.thestreet.com/technology/billionaire-adanis-empire-loses-51-billion-in-48-hours?puc=yahoo&cm_ven=YAHOO&yptr=yahoo,2023-01-28,"Suspicion is creating chaos in the empire built by the Indian billionaire Gautam Adani, who since last year has become the richest man in Asia. Adani, 60, saw his fortune jump by more than $40 bi...",6857


Preprocessing

In [165]:
def casefolding(sentence):
    return sentence.lower()
def cleaning(sentence):
    return re.sub(r'[^a-z]', ' ', re.sub("’", '', sentence))
def tokenization(sentence):
    return sentence.split()
def lemma(sentence):
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in sentence]
def remove_stopwords(sentence):
    stop_words = set(stopwords.words('english'))
    return [word for word in sentence if word not in stop_words]
def stemming(sentence):
    stemmer = nltk.PorterStemmer()
    return [stemmer.stem(word) for word in sentence]



In [166]:
df['text'] = df['text'].apply(casefolding)
df.head()

,title,link,published,text,text_length
0,"Chevron, Comcast, and 10 More Companies That Raised Their Dividends This Week",https://www.barrons.com/articles/chevron-comcast-blackrock-dividend-stocks-51674848026?siteid=yhoof2&yptr=yahoo,2023-01-28,,0
1,This janitor in Vermont amassed an $8M fortune without anyone around him knowing. Here are the 3 simple techniques that made Ronald Read rich — and can do the same for you,https://finance.yahoo.com/news/janitor-vermont-amassed-8m-fortune-140000770.html,2023-01-28,"warren buffett is reported to have once said, “you don't need to have extraordinary effort to achieve extraordinary results. you just need to do the ordinary, everyday things exceptionally well.” ...",4369
2,New 'PELOSI' bill may have a spiteful name — but there's still 'tremendous momentum' in the crusade to stop politicians from insider trading. Here's why,https://finance.yahoo.com/news/american-people-tired-lawmakers-introduce-150000544.html,2023-01-28,"in a mic drop moment in the debate around insider trading in congress, sen. josh hawley of missouri has introduced the pelosi act – trolling former speaker of the house nancy pelosi. with his acro...",5610
3,Lucid stock soars amid Saudi buyout speculation,https://finance.yahoo.com/news/lucid-stock-soars-amid-saudi-buyout-speculation-212055245.html,2023-01-28,lucid (lcid) shares soared as much as 88% on friday and were repeatedly halted for volatility following speculation that saudi arabia's public investment fund (pif) is considering buying a remaini...,2198
4,Billionaire Adani's Empire Loses $51 Billion in 48 Hours,https://www.thestreet.com/technology/billionaire-adanis-empire-loses-51-billion-in-48-hours?puc=yahoo&cm_ven=YAHOO&yptr=yahoo,2023-01-28,"suspicion is creating chaos in the empire built by the indian billionaire gautam adani, who since last year has become the richest man in asia. adani, 60, saw his fortune jump by more than $40 bi...",6857


In [167]:
df['text'] = df['text'].apply(cleaning)
df.head()

,title,link,published,text,text_length
0,"Chevron, Comcast, and 10 More Companies That Raised Their Dividends This Week",https://www.barrons.com/articles/chevron-comcast-blackrock-dividend-stocks-51674848026?siteid=yhoof2&yptr=yahoo,2023-01-28,,0
1,This janitor in Vermont amassed an $8M fortune without anyone around him knowing. Here are the 3 simple techniques that made Ronald Read rich — and can do the same for you,https://finance.yahoo.com/news/janitor-vermont-amassed-8m-fortune-140000770.html,2023-01-28,warren buffett is reported to have once said you don t need to have extraordinary effort to achieve extraordinary results you just need to do the ordinary everyday things exceptionally well ...,4369
2,New 'PELOSI' bill may have a spiteful name — but there's still 'tremendous momentum' in the crusade to stop politicians from insider trading. Here's why,https://finance.yahoo.com/news/american-people-tired-lawmakers-introduce-150000544.html,2023-01-28,in a mic drop moment in the debate around insider trading in congress sen josh hawley of missouri has introduced the pelosi act trolling former speaker of the house nancy pelosi with his acro...,5610
3,Lucid stock soars amid Saudi buyout speculation,https://finance.yahoo.com/news/lucid-stock-soars-amid-saudi-buyout-speculation-212055245.html,2023-01-28,lucid lcid shares soared as much as on friday and were repeatedly halted for volatility following speculation that saudi arabia s public investment fund pif is considering buying a remaini...,2198
4,Billionaire Adani's Empire Loses $51 Billion in 48 Hours,https://www.thestreet.com/technology/billionaire-adanis-empire-loses-51-billion-in-48-hours?puc=yahoo&cm_ven=YAHOO&yptr=yahoo,2023-01-28,suspicion is creating chaos in the empire built by the indian billionaire gautam adani who since last year has become the richest man in asia adani saw his fortune jump by more than bi...,6857


In [ ]:
# export to csv
df.to_csv('articles.csv', index=False)